# Recommendation Engine by Cosine similarity

Given simple dataset that collect text with labelled


*Last update: 26 Mar 2021, Part J.*


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
traindata = pd.read_csv('classifier_data/training_set.csv')

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from pythainlp.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
def wordToken(text):
    return word_tokenize(text, engine="newmm")

In [5]:
from easynmt import EasyNMT
model_nmt = EasyNMT('opus-mt')

def thnmt(text):
    if not text.isspace() and text:
        return model_nmt.translate(text, source_lang='th', target_lang='en')

## Without NMT

In [6]:
# Select only promotion

traindata.drop(traindata[traindata.bad == 1].index , inplace=True)

traindata.shape

(1061, 3)

In [7]:
#  check null
traindata.isnull().sum()

PromotionId      0
description    181
bad              0
dtype: int64

In [8]:
# This model will focus only text will remove empty description

traindata.dropna(inplace=True)

traindata.reset_index(drop=True, inplace=True)

traindata.shape

(880, 3)

In [9]:
# Clean up data and Extract Feature

traindata['feature'] = traindata.description

traindata['feature'] = traindata['feature'].replace('\n',' ', regex=True)

traindata['feature'] = traindata['feature'].replace('[^\w\s#@/:%.,_-]', '')

# token Thai word
traindata['feature'] = traindata['feature'].apply(wordToken)
traindata['feature'] = traindata['feature'].apply(' '.join)

cv = CountVectorizer()
features = cv.fit_transform(traindata['feature'])
print(cv.get_feature_names())
print(features.toarray())

['00', '000', '001', '011', '0111', '015', '0176182', '0189200', '02', '022124809', '022470123', '023224407', '0297', '03', '032', '043', '044', '052', '053', '060', '061', '0631408789', '064', '0649649363', '07', '076', '076604227', '08', '0820862388', '0825', '0828884237', '083', '084', '085', '086', '0866', '087', '0885555950', '089', '09', '090', '0925639939', '095', '0968602285', '097', '0974749900', '098', '10', '100', '1000', '105', '1073741828', '1080', '109', '11', '110', '1111', '11120', '115', '119', '12', '120', '1200', '121', '123', '124', '125', '127', '129', '1299', '13', '1300', '139', '14', '140', '145', '148', '149', '15', '150', '156', '157', '159', '16', '160', '1600', '1650', '168', '169', '17', '170', '171', '175', '179', '1790', '18', '180', '1800', '1810', '1818', '183', '185', '189', '19', '190', '1900', '195', '196', '199', '20', '200', '201', '2018', '203', '2038', '207', '209', '21', '210', '219', '22', '220', '225', '228', '229', '23', '230', '2352531571494

In [10]:
# Find cosine similarity
cosine_sim = cosine_similarity(features)

In [11]:
promotion = 1

similar_promotions = list(enumerate(cosine_sim[promotion]))

sorted_similar_promotions = sorted(similar_promotions,key=lambda x:x[1],reverse=True)[1:]

In [12]:
# Test similarity

print(traindata['feature'][promotion])
print("------------")

i=0
for element in sorted_similar_promotions:
    print(element)
    print(traindata['feature'][element[0]])
    i=i+1
    if i>3:
        break

ทะเล ทะลัก !!   สุดยอด บุฟเฟ่ต์ ทะเล   ที่ คุณ ไม่ ควร พลาด        ลด สูงสุด   60 %   จาก ราคา   1900   บาท สุทธิ !   จอง เลย     วัน ศุกร์ และ วัน เสาร์   ตั้งแต่ เวลา   18.00 - 22.00   น.     ราคา นี้ รวม ชา   กาแฟร้อน ไม่ อั้น   น้ำอัดลม     สมาชิก แอคคอร์ พลัส รับ ส่วนลด ออน ท็อป อีก   10 %
------------
(80, 0.8981462390204991)
ทะเล ทะลัก !!   สุดยอด บุฟเฟ่ต์ ทะเล   ที่ คุณ ไม่ ควร พลาด        ฟิน กับ   บฟเฟต์ ทะเล   ที่ ขน ปูม้า   ปูทะเล   หอยนางรม   กุ้ง แม่น้ำ   กั้งกระดาน   และ อีก มากมาย มา ให้ อร่อย   ลด สูงสุด   60 %   จาก ราคา   1900   บาท สุทธิ !   จอง เลย     ***** วัน ศุกร์ และ วัน เสาร์   ตั้งแต่ เวลา   18.00 - 22.00   น.     ราคา นี้ รวม ชา   กาแฟร้อน ไม่ อั้น   น้ำอัดลม     สมาชิก แอคคอร์ พลัส รับ ส่วนลด ออน ท็อป อีก   10
(220, 0.6500000000000001)
จอง ตรง ผ่าน เว็บไซต์   ลด สูงสุด   60 %   จาก   1,530   บาท สุทธิ !        บุฟเฟ่ต์ ปู ! กุ้ง !   จัด เต็ม   แบบ ไม่ อั้น      ทุก อาทิตย์   -   พฤหัสบดี   765   บาท ถ้วน !!        ทุก วันอาทิตย์ - พฤหัสบดี   ตั้งแต่ เวลา  

## With NMT

In [13]:
# resue NMT file from spam classifier

nmt_traindata = pd.read_csv('nmt-full.csv')
nmt_traindata.isnull().sum()
nmt_traindata.fillna('empty', inplace = True)
nmt_traindata.isnull().sum()
nmt_traindata.head()

,PromotionId,description,bad,feature_en_nmt
0,1NWLBO3bwOUOB88Sr3GAo31nbptCWW,โปรโมชั่นเปิดร้าน วันนี้วันเดียว\n\n\n\n18 08 ...,0,"It's opening today. One day, 1808 18. ..."
1,1NWLBqv7ODub2WEc1kjDD4h5DEdal1,สมาชิก,0,Members
2,1NWLBqu73sYzuFXL4DJsq4gw6cxU10,โปรโมชั่น “Live Canadian Lobster”กับ 3 เมนูสุด...,0,The promotion “Live Canadian Lobster ” and 3 m...
3,1NWLCcUaQzZ0eeDpZ6Y68wpjSaynFg,เมนูเซ็ต ราคาพิเศษ พร้อมบริการส่งถึงที่,0,Special Price Set Menu at Hand
4,1NWLAZSnaHLTkubTkprNOmdOcQT23l,empty,1,empty


In [14]:
cv = CountVectorizer()
features = cv.fit_transform(nmt_traindata.feature_en_nmt)
print(cv.get_feature_names())
print(features.toarray())


['00', '000', '002', '015', '02', '022', '023224407', '033', '043', '044', '05', '052', '05399333', '054', '062', '0631408789', '0649492665', '0649649363', '069', '07', '076', '08', '080', '081', '082', '0820862388', '082828', '083', '085', '086', '0866', '0885555950', '089', '0894184489', '09', '090', '0925639', '094', '095', '0968602285', '097', '098', '099', '10', '100', '1000', '1010', '102030', '109', '10th', '11', '110', '1100', '119', '11th', '12', '120', '121', '122', '123', '1234', '124', '125', '127', '129', '12th', '13', '13th', '14', '140', '1400', '1423313', '145', '145780235538933', '148', '149', '14th', '15', '150', '1500', '157', '159', '15th', '16', '1600', '1650', '17', '170', '1700', '171', '175', '179', '17th', '18', '180', '1800', '1808', '181', '188', '189', '18th', '19', '190', '1920', '1951', '196', '199', '19th', '1g', '1r', '1st', '20', '200', '2011', '2013', '2018', '203', '2038', '2056', '208', '209', '20th', '20um', '21', '210', '2100', '2130', '216', '219'

In [15]:
cosine_sim = cosine_similarity(features)

In [16]:
# Test similarity
promotion = 2

similar_promotions = list(enumerate(cosine_sim[promotion]))

sorted_similar_promotions = sorted(similar_promotions,key=lambda x:x[1],reverse=True)[1:]

print(nmt_traindata['feature_en_nmt'][promotion])
print("------------")

i=0
for element in sorted_similar_promotions:
    print(element)
    print(nmt_traindata['feature_en_nmt'][element[0]])
    i=i+1
    if i>3:
        break

The promotion “Live Canadian Lobster ” and 3 menus. Hong Kong’s hit - and - miss style at a special price Only 2,100 + 100, Hong Kong Fisherman store, building 8 empansity center, 833 5434 - 5,02 833 3637 37 39 service every day at 11: 00 - 22 p.m. Last year’s issue of the Watch Tower Bible and Tract Society of New York, U.S.A.
------------
(307, 0.47023329256650215)
  It's a precipitation for the rainy season and the set of African tea.    สำหรับ   2 Ready for the price of $1.5 a.m., between 12: 00 p.m. and 7:00 p.m., the president of the Watch Tower Bible and Tract Society of New York, U.S.A., received a letter from the branch office of Jehovah’s Witnesses in Brooklyn, New York.  
(1369, 0.42447635997800887)
"Tamago Spectal promotion at a special price and a 20% discount."   สำหรับ None of the other menus (except drinks and desserts) Plus Vet and Service Care.
(286, 0.4086942641998112)
The Bible says: “The word of God is alive and exerts power and is sharper than any two - edged swor